### Fitting Naive Bayes to the Training set

posterior = prior occurrances * liklihood / evidence
The "gaussian" and "naive" come from two assumptions present in this likelihood:
1. we assume each feature is uncorrelated from each other. This is obviously not true, and is a "naive"                assumption.
2. we assume have that the value of the features (e.g. meannfreq of women) are normally          (gaussian) distributed.

In [39]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd 
import numpy as np  
import random
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split  
from collections import defaultdict
from sklearn.model_selection import cross_val_score

In [40]:
%store -r train_x
%store -r test_x
%store -r train_y
%store -r test_y

%store -r train_x_two_features
%store -r test_x_two_features
%store -r train_y_two_features
%store -r test_y_two_features

%store -r pca_train_x
%store -r pca_test_x

In [41]:
def run_naive_bayes(train_x, test_x, train_y, test_y):
    clf = GaussianNB()
    clf.fit(train_x, train_y)
    scores = cross_val_score(clf, train_x, train_y, cv=5)
    print("In-sample accuracy for svm with Linear kernel: %.10f" % scores.mean())
    scores = cross_val_score(clf, test_x, test_y, cv=5)
    print("Out-of-sample accuracy for svm with Linear kernel: %.10f" % scores.mean())

In [49]:
# # For both the classes 0 and 1, and for each of the 20 features, we have thus saved the mean and the standard in summary. 
# # For example, for the class 0 (female), and feature 1 (ie, sd), mean and the standard for this attribute are 0.060910268026077803 and 0.00034668419803097784

# def getSummary(data,n_cls):
#     data_means = data.groupby('label').mean()
#     data_variance = data.groupby('label').var()
    
#     cnames = np.array(data.columns.tolist())[:-1] 
#     classes = np.unique(labels)
#     classnames = np.array(['female','male'])
#     summary = {}

#     for cls in classes:
#         #initializing the dictionary
#         summary[cls] = defaultdict(list)
#     for cls in classes:
#         for j in range(0, n_cls):
#             print(classnames[cls])
#             summary[cls][j] += list(np.array([data_means[cnames[j]][data_means.index == classnames[cls]].values[0],
#                                           data_variance[cnames[j]][data_variance.index == classnames[cls]].values[0]]))
            
#     print summary   
#     return summary

In [50]:
# summary = getSummary(train_df,train_x.shape[1])

In [51]:
# # Group the data by gender and calculate the means of each feature
# data_means = train_x_df.groupby('label').mean()
# # View the values
# data_means

In [52]:
# # Group the data by gender and calculate the variance of each feature
# data_variance = train_x_df.groupby('label').var()
# # View the values
# data_variance

In [53]:
# # Create a function to calculate the probability density of each of the terms of the likelihood 
# # We use mean and standard deviation of input values (x) for each class to summarize the distribution
# # Probabilities of new x values are calculated using the Gaussian Probability Density Function (PDF).

# def p_x_given_y(x, mean_y, variance_y):
#     # Input the arguments into a probability density function
#     p = 1/(np.sqrt(2*np.pi*variance_y)) * np.exp((-(x-mean_y)**2)/(2*variance_y))
#     return p

# # We calculate the probability that the value of the random variable will be between x - 0.01 and x + 0.01


In [54]:
# # Test
# p_x_given_y(2, 2, 4)

In [55]:
# def calculateClassProbabilities(summaries, inputVector):
#     # p(x|y) (e.g. p(meannfreq∣female))
#     probabilities = {}
#     for classValue, classSummaries in summaries.iteritems():
#         if (classValue == 0):
#             probabilities[classValue] = P_female
        
#         else:
#             probabilities[classValue] = P_male
#         for i in range(len(classSummaries)):
#             mean, var = classSummaries[i]
#             x = inputVector[i]
#             probabilities[classValue] *= p_x_given_y(x, mean, var)
#     return probabilities

In [56]:
# calculateClassProbabilities(summary, test_x[10])
# # PDF can be greater than 1, but it can't be greater than 1 for a large interval
# # because of the non-negativity and integral constraints.

In [57]:
# # Make a Prediction
# import operator
# def predict(summaries, inputVector):
#     probabilities = calculateClassProbabilities(summaries, inputVector)
#     #print probabilities
#     probabilities = sorted(probabilities.iteritems(), key=operator.itemgetter(1), reverse=True)
#     return probabilities[0][0]